In [ ]:
import os
import os.path
import sys
from pathlib import Path
sys.path.insert(0, '..')
from elitetools import eddb
eddb.load_feeds()

In [ ]:
origin_name="Dvorsi"
commodities = "Tobacco, Gold, Mineral Oil"
minimum_supply = 128
top_count = 5

In [ ]:
commodity_names=[ c.strip() for c in commodities.split(',') ]
nearby_sources = eddb.commodity_sources_nearby(origin_name, commodity_names, minimum_supply, top_count)
nearby_sources